# Google BFS question

In [1]:
from collections import deque

Given a 2D board with emergency $E$ and some policeman $P$, what is the distance from an emergency to closest policeman. Nodes marked as $X$, are walls and cannot be traversed.


Think a little bit about how to solve it

In [66]:
example = [
    "...P...",
    "...XXX.",
    "EX..P..",
    ".X.....",
    ".......",
    ".......",
]

class MapGraph(object):
    def __init__(self, mmap):
        """Graph abstraction for our map"""
        self.dim_x, self.dim_y = len(mmap), len(mmap[0])
        self.mmap = {(x,y):mmap[x][y] for x in range(self.dim_x) 
                                      for y in range(self.dim_y)}

    def neighbours(self, node):
        """Returns all the direct neighbors of a given node"""
        x,y = node
        # There are four directions in which we can go
        for dx, dy in [[0,1], [0,-1], [-1,0], [1,0]]:
            nx, ny = x + dx, y + dy
            if (0 <= nx < self.dim_x and # but we cannot got through walls
                    0 <= ny < self.dim_y # (at least not yet!)
                    and self.mmap[nx,ny] != 'X'):
                yield nx, ny
    
    def find_all(self, letter):
        """Finds all the coordinates where a given letter occurs"""
        res = []
        for x in range(self.dim_x):
            for y in range(self.dim_y):
                if self.mmap[x,y] == letter:
                    res.append((x,y))
        return res
    
    def show(self, what=None):
        """Displays the graph"""
        what = what or self.mmap
        for x in range(self.dim_x):
            for y in range(self.dim_y):
                print(what[x,y] if (x,y) in what else '?', end='')
                print(' ', end='')
            print('')

In [67]:
g = MapGraph(example)
g.show()
g.find_all('E')

. . . P . . . 
. . . X X X . 
E X . . P . . 
. X . . . . . 
. . . . . . . 
. . . . . . . 


[(2, 0)]

The solution is maybe somewhat counterintuitive - we start our search from emergency, not policeman

In [68]:
def bfs_from(graph, source):
    q = deque()
    # initially source is on the queue
    distance = {source: 0}
    q.appendleft(source)
    # while queue is not empty
    while len(q) > 0:
        # consider the node that has been in the queue for
        # the longest
        node = q.popleft()
        # for all neighbours
        for neighbour in graph.neighbours(node):
            # if they were NOT visted yet
            if neighbour not in distance:
                # mark their distance and put them on queue
                distance[neighbour] = distance[node] + 1
                q.append(neighbour)
    return distance

In [69]:
g = MapGraph(example2)
distances = bfs_from(g, g.find_all('E')[0])
g.show()
print('')
g.show(distances)

. . . P . . . 
. . . X X X . 
E X . . P . . 
. X . . . . . 
. . . . . . . 
. . . . . . . 

2 3 4 5 6 7 8 
1 2 3 ? ? ? 9 
0 ? 4 5 6 7 8 
1 ? 5 6 7 8 9 
2 3 4 5 6 7 8 
3 4 5 6 7 8 9 


In [70]:
def solve_google(example):
    g = MapGraph(example)
    # comute distances from emergency to everywhere welse
    distances = bfs_from(g, g.find_all('E')[0])
    # find the minimum distance policeman
    res = float('inf')
    for policeman in g.find_all('P'):
        res = min(res, distances[policeman])
    return res

In [71]:
solve_google(example)

5

## Success! 

We solve the problem correctly in $O(nm)$, which is the best solution we can hoped for!

## Follow up question

Find the worst place for emergency (furthest from all policeman)

Think about how to solve it.

In [72]:
example2 = [
    "...P...",
    "...XXX.",
    "EX..P..",
    ".X.....",
    ".......",
    ".......",
]

There are many suboptimal solutions, but it turns out we can still solve it in $O(nm)$, by starting BFS from all the policeman simultaneoursly!

In [73]:
def bfs_from_many(graph, sources):
    q = deque()
    distance = {s: 0 for s in sources} # <--- this line changed
    q.extendleft(sources)              # <--- this line changed
    while len(q) > 0:
        node = q.popleft()
        for neighbour in graph.neighbours(node):
            if neighbour not in distance:
                distance[neighbour] = distance[node] + 1
                q.append(neighbour)
    return distance

In [74]:
g = MapGraph(example2)
distances = bfs_from_many(g, g.find_all('P'))
g.show()
print('')
g.show(distances)

. . . P . . . 
. . . X X X . 
E X . . P . . 
. X . . . . . 
. . . . . . . 
. . . . . . . 

3 2 1 0 1 2 3 
4 3 2 ? ? ? 3 
5 ? 2 1 0 1 2 
6 ? 3 2 1 2 3 
6 5 4 3 2 3 4 
7 6 5 4 3 4 5 


In [60]:
def solve_google_hard(example):
    g = MapGraph(example)
    # compute distances from all the policeman
    distances = bfs_from_many(g, g.find_all('E')[0])
    # find the minimum distance policeman
    return max(distances.values())

In [75]:
solve_google_hard(example2)

TypeError: 'int' object is not iterable